# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as articles, social media posts, product descriptions, and more. This can help reduce content creation costs and increase efficiency.
2. **Image and Video Generation**: Generative AI can create realistic images and videos for use in marketing campaigns, advertising, and product demonstrations.
3. **Customer Service Chatbots**: Generative AI-powered chatbots can be used to provide customer support, answering frequently asked questions, and helping customers with simple issues.
4. **Personalized Recommendations**: Generative AI algorithms can analyze customer behavior and preferences to generate personalized recommendations for products, services, or content.
5. **Product Design and Prototyping**: Generative AI can aid in product design and prototyping by generating multiple design options, reducing the need for physical pro

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as blog posts, social media posts, product descriptions, and even entire articles. This can help businesses save time and resources while still producing engaging and relevant content.
2. **Product Design**: Generative AI can assist in the design of products by generating 3D models, prototypes, and even functional prototypes. This can be particularly useful for companies that require rapid prototyping or have limited in-house design capabilities.
3. **Marketing Automation**: Generative AI can help automate marketing efforts by generating personalized email campaigns, social media ads, and even lead generation content.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answering common queries, and helping to resolve simple issues. This can free up h

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [14]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [33]:
# Constants
MODEL = "llama3.2"

In [34]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [35]:
ed = Website("https://jaivikhimalay.com")
print(ed.url)
print(ed.title)
print(ed.text)

https://jaivikhimalay.com
Jaivik Himalay
0
Skip to Content
Services
About
Contact
Open Menu
Close Menu
Services
About
Contact
Open Menu
Close Menu
Services
About
Contact
Enhance the accessibility of your products and services to customers
Application Architecture, Design & Development Services
Engaging and solving customers’ requirements to automate their business processes, by transforming requirements into a ready to use products.
Application Refactoring, Performance Improvement & Integration Services
Keeping clients by leveraging the latest practices and technologies in the industry to transform their outdated and sluggish systems into ones that are incredibly efficient.
Distribution Services
Assisting companies in meeting their requirements by sourcing a diverse range of products from both local and global markets.
Read More
Services
Application Architecture, Design & Development Services
We offer support to small and medium-sized businesses in developing their products or APIs fro

In [36]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [37]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [38]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [39]:
# And now: call the Ollama function instead of OpenAI

def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [40]:
summarize("https://jaivikhimalay.com")

"# Website Summary\n### Overview\n\nJaivik Himalay is a technology consulting firm that offers a range of services to help businesses improve their products and services, automate business processes, and enhance accessibility.\n\n### Services\n\n* **Application Architecture**: Development of applications from concept to launch for small and medium-sized businesses.\n* **Refactoring & Integration**: Transformation of outdated applications into efficient ones using the latest industry practices and technologies.\n* **Distribution Services**: Sourcing diverse products from local and global markets to meet clients' requirements.\n\n### About Us\n\nJaivik Himalay is a group of seasoned experts with 20 years of experience solving complex issues in various areas, including payments, gift cards, logistics. The company is based in California's Silicon Valley.\n\n### Contact Us\n\nThe company invites interested individuals to contact them through the provided email address."

In [41]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [42]:
display_summary("https://jaivikhimalay.com")

# Jaivik Himalaya Website Summary

## Overview

Jaivik Himalaya is a professional services company specializing in application development, refactoring, and distribution services.

### Key Services

- **Application Architecture**: Designing and developing products or APIs from concept to launch.
- **Application Refactoring & Performance Improvement**: Transforming outdated systems into efficient ones using latest industry practices and technologies.
- **Distribution Services**: Sourcing products from local and global markets to assist businesses in meeting their requirements.

### About Us

Jaivik Himalaya is led by a group of experienced experts with 20 years of experience solving complex issues in various domains such as payments, gift cards, logistics, etc. The company is based in Silicon Valley, California.

### Contact Information

- Email: `info@jaivikhimalay.com`
- Location: Dublin, CA 94568
- Address: No specific address provided on the website.